## Exercise: Using LSTMs to Classify the 20 Newsgroups Data Set
The 20 Newsgroups data set is a well known classification problem. The goal is to classify which newsgroup a particular post came from.  The 20 possible groups are:

`comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x	rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey	
sci.crypt
sci.electronics
sci.med
sci.space
misc.forsale	
talk.politics.misc
talk.politics.guns
talk.politics.mideast	
talk.religion.misc
alt.atheism
soc.religion.christian`

As you can see, some pairs of groups may be quite similar while others are very different.

The data is given as a designated training set of size 11314 and test set of size 7532.  The 20 categories are represented in roughly equal proportions, so the baseline accuracy is around 5%.


To begin, review the code below.  This will walk you through the basics of loading in the 20 newsgroups data, loading in the GloVe data, building the word embedding matrix, and building the LSTM model.

After we build the first LSTM model, it will be your turn to build one and play with the parameters.

In [1]:
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

import keras

from sklearn.datasets import fetch_20newsgroups

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
max_features = 20000
seq_length = 30  # How long to make our word sequences
batch_size = 32



In [3]:
# Download the 20 newsgroups data - there is already a designated "train" and "test" set

newsgroup_train = fetch_20newsgroups(subset='train')
newsgroup_test = fetch_20newsgroups(subset='test')


In [4]:
len(newsgroup_train.data), len(newsgroup_test.data)

(11314, 7532)

We will be using the Glove pre-trained word vectors.  If you haven't already, please download them using this link:
(NOTE: this will start downloading an 822MB file)

http://nlp.stanford.edu/data/glove.6B.zip

Then unzip the file and fill your local path to the file in the code cell below.

We will use the file `glove.6B.100d.txt`

In [5]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(newsgroup_train.data)

Let's just look at a word embedding

In [6]:
sequences_train = tokenizer.texts_to_sequences(newsgroup_train.data)
sequences_test = tokenizer.texts_to_sequences(newsgroup_test.data)

In [7]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 134142 unique tokens.


In [8]:
x_train = pad_sequences(sequences_train, maxlen=seq_length)
x_test = pad_sequences(sequences_test, maxlen=seq_length)

In [9]:
y_train = keras.utils.to_categorical(np.asarray(newsgroup_train.target))
y_test = keras.utils.to_categorical(np.asarray(newsgroup_test.target))

In [10]:
embeddings_index = {}

f = open('C:/Users/lucas/Documents/GitHub/glove.6B/glove.6B.100d.txt', encoding='utf-8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [11]:
dog_vec = embeddings_index['dog']
dog_vec

array([ 0.30817  ,  0.30938  ,  0.52803  , -0.92543  , -0.73671  ,
        0.63475  ,  0.44197  ,  0.10262  , -0.09142  , -0.56607  ,
       -0.5327   ,  0.2013   ,  0.7704   , -0.13983  ,  0.13727  ,
        1.1128   ,  0.89301  , -0.17869  , -0.0019722,  0.57289  ,
        0.59479  ,  0.50428  , -0.28991  , -1.3491   ,  0.42756  ,
        1.2748   , -1.1613   , -0.41084  ,  0.042804 ,  0.54866  ,
        0.18897  ,  0.3759   ,  0.58035  ,  0.66975  ,  0.81156  ,
        0.93864  , -0.51005  , -0.070079 ,  0.82819  , -0.35346  ,
        0.21086  , -0.24412  , -0.16554  , -0.78358  , -0.48482  ,
        0.38968  , -0.86356  , -0.016391 ,  0.31984  , -0.49246  ,
       -0.069363 ,  0.018869 , -0.098286 ,  1.3126   , -0.12116  ,
       -1.2399   , -0.091429 ,  0.35294  ,  0.64645  ,  0.089642 ,
        0.70294  ,  1.1244   ,  0.38639  ,  0.52084  ,  0.98787  ,
        0.79952  , -0.34625  ,  0.14095  ,  0.80167  ,  0.20987  ,
       -0.86007  , -0.15308  ,  0.074523 ,  0.40816  ,  0.0192

In [12]:
## This creates a matrix where the $i$th row gives the word embedding for the word represented by integer $i$.
## Essentially, these will be the "weights" for the Embedding Layer
## Rather than learning the weights, we will use these ones and "freeze" the layer

embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [13]:
embedding_matrix.shape

(134143, 100)

## LSTM Layer
`keras.layers.recurrent.LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- Similar in structure to the `SimpleRNN` layer
- `units` defines the dimension of the recurrent state
- `recurrent_...` refers the recurrent state aspects of the LSTM
- `kernel_...` refers to the transformations done on the input



In [16]:
word_dimension = 100

model = Sequential()
model.add(Embedding(len(word_index) + 1, 
                    word_dimension, 
                    weights=[embedding_matrix], 
                    input_length=seq_length, 
                    trainable=False))

model.add(LSTM(30, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(20, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 100)           13414300  
_________________________________________________________________
lstm (LSTM)                  (None, 30)                15720     
_________________________________________________________________
dense (Dense)                (None, 20)                620       
Total params: 13,430,640
Trainable params: 16,340
Non-trainable params: 13,414,300
_________________________________________________________________


In [17]:
rmsprop = keras.optimizers.RMSprop(lr=0.002)

model.compile(loss='categorical_crossentropy', 
              optimizer=rmsprop, 
              metrics=['accuracy'])

In [18]:
model.fit(x_train, y_train, 
          batch_size=batch_size, 
          epochs=20, 
          validation_data=(x_test, y_test))

Epoch 1/20
354/354 [==============================] - 7s 21ms/step - loss: 2.7262 - accuracy: 0.1577 - val_loss: 2.4701 - val_accuracy: 0.2257
Epoch 2/20
354/354 [==============================] - 8s 23ms/step - loss: 2.2822 - accuracy: 0.2882 - val_loss: 2.1603 - val_accuracy: 0.3201
Epoch 3/20
354/354 [==============================] - 10s 27ms/step - loss: 2.0549 - accuracy: 0.3577 - val_loss: 2.0599 - val_accuracy: 0.3521
Epoch 4/20
354/354 [==============================] - 9s 26ms/step - loss: 1.9154 - accuracy: 0.4084 - val_loss: 1.9449 - val_accuracy: 0.3898
Epoch 5/20
354/354 [==============================] - 9s 25ms/step - loss: 1.8205 - accuracy: 0.4354 - val_loss: 2.0093 - val_accuracy: 0.3805
Epoch 6/20
354/354 [==============================] - 9s 26ms/step - loss: 1.7400 - accuracy: 0.4639 - val_loss: 1.8826 - val_accuracy: 0.4218
Epoch 7/20
354/354 [==============================] - 9s 26ms/step - loss: 1.6800 - accuracy: 0.4827 - val_loss: 1.8597 - val_accuracy: 0.428

## Exercise
### Your Turn
- Build a neural network with a SimpleRNN instead of an LSTM (with other dimensions and parameters the same). How does the performance compare?
- Use the LSTM above without the pretrained word vectors (randomly initialize the weights and have them be learned during the training process).  How does the performance compare?
- Try different sequence lengths, and dimensions for the hidden state of the LSTM.  Can you improve the model?


In [19]:
from keras.layers import SimpleRNN
from keras import initializers

In [20]:
rnn_hidden_dim = 50

model_2 = Sequential()
model_2.add(Embedding(len(word_index) + 1, 
                    100, 
                    weights=[embedding_matrix], 
                    input_length=seq_length, 
                    trainable=False))

model_2.add(SimpleRNN(rnn_hidden_dim, kernel_initializer=initializers.RandomNormal(stddev=0.001), 
                      recurrent_initializer=initializers.Identity(gain=1.0), 
                      activation='relu'))

model_2.add(Dense(20, activation='softmax'))

model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 100)           13414300  
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 50)                7550      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1020      
Total params: 13,422,870
Trainable params: 8,570
Non-trainable params: 13,414,300
_________________________________________________________________


In [21]:
rmsprop = keras.optimizers.RMSprop(lr=0.002)

model_2.compile(loss='categorical_crossentropy', 
              optimizer=rmsprop, 
              metrics=['accuracy'])

In [22]:
model_2.fit(x_train, y_train, 
          batch_size=batch_size, 
          epochs=100, 
          validation_data=(x_test, y_test))

Epoch 1/100
354/354 [==============================] - 2s 7ms/step - loss: 2.8911 - accuracy: 0.1754 - val_loss: 2.8559 - val_accuracy: 0.2410
Epoch 2/100
354/354 [==============================] - 2s 5ms/step - loss: 2.3090 - accuracy: 0.3055 - val_loss: 2.1997 - val_accuracy: 0.3328
Epoch 3/100
354/354 [==============================] - 2s 5ms/step - loss: 2.1286 - accuracy: 0.3568 - val_loss: 2.2089 - val_accuracy: 0.3286
Epoch 4/100
354/354 [==============================] - 2s 4ms/step - loss: 2.0135 - accuracy: 0.3895 - val_loss: 2.1596 - val_accuracy: 0.3428
Epoch 5/100
354/354 [==============================] - 2s 5ms/step - loss: 1.9222 - accuracy: 0.4109 - val_loss: 2.0930 - val_accuracy: 0.3699
Epoch 6/100
354/354 [==============================] - 2s 4ms/step - loss: 1.8715 - accuracy: 0.4281 - val_loss: 2.1908 - val_accuracy: 0.3698
Epoch 7/100
354/354 [==============================] - 2s 4ms/step - loss: 1.8299 - accuracy: 0.4444 - val_loss: 2.2570 - val_accuracy: 0.3781

Epoch 58/100
354/354 [==============================] - 2s 6ms/step - loss: 1.3833 - accuracy: 0.5844 - val_loss: 3.0578 - val_accuracy: 0.3918
Epoch 59/100
354/354 [==============================] - 2s 6ms/step - loss: 1.3940 - accuracy: 0.5837 - val_loss: 3.8533 - val_accuracy: 0.3894
Epoch 60/100
354/354 [==============================] - 2s 5ms/step - loss: 1.4357 - accuracy: 0.5760 - val_loss: 3.8640 - val_accuracy: 0.3856
Epoch 61/100
354/354 [==============================] - 2s 6ms/step - loss: 1.3854 - accuracy: 0.5808 - val_loss: 4.9322 - val_accuracy: 0.3958
Epoch 62/100
354/354 [==============================] - 2s 6ms/step - loss: 1.6968 - accuracy: 0.5812 - val_loss: 3.1243 - val_accuracy: 0.3864
Epoch 63/100
354/354 [==============================] - 2s 5ms/step - loss: 1.3848 - accuracy: 0.5814 - val_loss: 3.0609 - val_accuracy: 0.3909
Epoch 64/100
354/354 [==============================] - 2s 6ms/step - loss: 1.4219 - accuracy: 0.5848 - val_loss: 3.1603 - val_accuracy:

### Build a neural network with a SimpleRNN instead of an LSTM (with other dimensions and parameters the same). How does the performance compare?

While model_2 seems to learn the training data better than our original model.  However, the original mode performs better on the testing data showing it generalizes better to unforseen data.

In [25]:
model_3 = Sequential()
model_3.add(Embedding(len(word_index) + 1, 
                    word_dimension,
                    trainable=False))

model_3.add(LSTM(30, 
                 dropout=0.2, 
                 recurrent_dropout=0.2))

model_3.add(Dense(20, activation='softmax'))

model_3.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 100)         13414300  
_________________________________________________________________
lstm_3 (LSTM)                (None, 30)                15720     
_________________________________________________________________
dense_4 (Dense)              (None, 20)                620       
Total params: 13,430,640
Trainable params: 16,340
Non-trainable params: 13,414,300
_________________________________________________________________


In [26]:
rmsprop = keras.optimizers.RMSprop(lr=0.002)

model_3.compile(loss='categorical_crossentropy', 
              optimizer=rmsprop, 
              metrics=['accuracy'])

In [27]:
model_3.fit(x_train, y_train, 
          batch_size=batch_size, 
          epochs=20, 
          validation_data=(x_test, y_test))

Epoch 1/20
354/354 [==============================] - 7s 20ms/step - loss: 2.9808 - accuracy: 0.0714 - val_loss: 2.9590 - val_accuracy: 0.0834
Epoch 2/20
354/354 [==============================] - 8s 22ms/step - loss: 2.9188 - accuracy: 0.1023 - val_loss: 2.9201 - val_accuracy: 0.0971
Epoch 3/20
354/354 [==============================] - 9s 26ms/step - loss: 2.8625 - accuracy: 0.1180 - val_loss: 2.8856 - val_accuracy: 0.1183
Epoch 4/20
354/354 [==============================] - 9s 26ms/step - loss: 2.8267 - accuracy: 0.1354 - val_loss: 2.8508 - val_accuracy: 0.1240
Epoch 5/20
354/354 [==============================] - 9s 25ms/step - loss: 2.7955 - accuracy: 0.1486 - val_loss: 2.8326 - val_accuracy: 0.1418
Epoch 6/20
354/354 [==============================] - 9s 25ms/step - loss: 2.7645 - accuracy: 0.1621 - val_loss: 2.8130 - val_accuracy: 0.1386
Epoch 7/20
354/354 [==============================] - 9s 26ms/step - loss: 2.7355 - accuracy: 0.1731 - val_loss: 2.8086 - val_accuracy: 0.1443

### Use the LSTM above without the pretrained word vectors (randomly initialize the weights and have them be learned during the training process). How does the performance compare?

The accuracies for both training and testing data are significantly lower.

## Try different sequence lengths, and dimensions for the hidden state of the LSTM. Can you improve the model?

In [26]:
model_4 = Sequential()
model_4.add(Embedding(len(word_index) + 1, 
                    100, 
                    weights=[embedding_matrix], 
                    input_length=50, 
                    trainable=False))

model_4.add(LSTM(100, # 50 
                 dropout=0.3, 
                 recurrent_dropout=0.3))

model_4.add(Dense(20, activation='softmax'))

model_4.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 50, 100)           13414300  
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 20)                2020      
Total params: 13,496,720
Trainable params: 82,420
Non-trainable params: 13,414,300
_________________________________________________________________


In [27]:
rmsprop = keras.optimizers.RMSprop(lr=0.002)

model_4.compile(loss='categorical_crossentropy', 
              optimizer=rmsprop, 
              metrics=['accuracy'])

In [28]:
model_4.fit(x_train, y_train, 
          batch_size=batch_size, 
          epochs=20, 
          validation_data=(x_test, y_test))

Epoch 1/20
354/354 [==============================] - 10s 29ms/step - loss: 2.5567 - accuracy: 0.2021 - val_loss: 2.2258 - val_accuracy: 0.2921
Epoch 2/20
354/354 [==============================] - 12s 35ms/step - loss: 2.0785 - accuracy: 0.3541 - val_loss: 2.0825 - val_accuracy: 0.3522
Epoch 3/20
354/354 [==============================] - 12s 35ms/step - loss: 1.8314 - accuracy: 0.4317 - val_loss: 1.8374 - val_accuracy: 0.4331
Epoch 4/20
354/354 [==============================] - 12s 35ms/step - loss: 1.6645 - accuracy: 0.4842 - val_loss: 1.7980 - val_accuracy: 0.4546
Epoch 5/20
354/354 [==============================] - 12s 35ms/step - loss: 1.5464 - accuracy: 0.5202 - val_loss: 1.7525 - val_accuracy: 0.4744
Epoch 6/20
354/354 [==============================] - 12s 35ms/step - loss: 1.4479 - accuracy: 0.5535 - val_loss: 1.7581 - val_accuracy: 0.4790
Epoch 7/20
354/354 [==============================] - 12s 34ms/step - loss: 1.3692 - accuracy: 0.5816 - val_loss: 1.7336 - val_accuracy:

By increase the our sequence length to 50 and building 100 hidden layers we have a model that in theory is better than a random guess, unlike the other models.  However, when comparing the training data accuracy with validation data we can see a strong indicator of overfitting the training set in our model.  